In [2]:
#CREDITS https://jackmckew.dev/sentiment-analysis-text-cleaning-in-python-with-vader.html
#on code for consolidating top sentiments in one video dataset
#then modify code to consolidate top videos with highest overall positive sentiments instead

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import pandas as pd
import json

analyzer = SentimentIntensityAnalyzer()




In [2]:
video_id = '_O2sg-PGhEg' #video id of df1 in get_sentiment_vader.ipynb 
#text_data = pd.read_table('original_rt_snippets.txt',header=None) 

text_data = pd.read_json(f'../results/crashcourse/{video_id}.json')

In [3]:
def get_sentiment(text:str, analyserobj,desired_type:str='pos'):
    # Get sentiment from text
    sentiment_score = analyserobj.polarity_scores(text)
    return sentiment_score[desired_type]


# Get Sentiment scores
def get_sentiment_scores(df,data_column):
    sentimentlist = [] #empty list should be here because if looped through all videos of a channel, sentimentlist should renew
    
    #df[f'{data_column} negative'] = df[data_column].astype(str).apply(lambda x: get_sentiment(x,analyzer,'neg'))
    df['negative'] = df[data_column].astype(str).apply(lambda x: get_sentiment(x,analyzer,'neg'))
    df['neutral'] = df[data_column].astype(str).apply(lambda x: get_sentiment(x,analyzer,'neu'))
    df['positive'] = df[data_column].astype(str).apply(lambda x: get_sentiment(x,analyzer,'pos'))
    df['compound'] = df[data_column].astype(str).apply(lambda x: get_sentiment(x,analyzer,'compound'))
    
    for comp in df['compound']:
        if comp >= 0.05:
            sentiment = 'positive'
        elif comp <= -0.05:
            sentiment = 'negative'
        else:
            sentiment = 'neutral'
            
        sentimentlist.append(sentiment)

    df['sentiment'] = sentimentlist
    
    return df


In [4]:
text_sentiment = get_sentiment_scores(text_data, 'comment_text')


In [5]:
text_sentiment

,comment_by,comment_text,comment_date,likes_count,negative,neutral,positive,compound,sentiment
0,sam JD,awesome host.. thank u so much,2022-07-14T09:38:00Z,1,0.000,0.377,0.623,0.7650,positive
1,Franzi Z,I love this series more with every episode I w...,2020-06-18T17:26:03Z,4,0.000,0.499,0.501,0.9440,positive
2,Wade Allen,When I see images or renderings of super clust...,2020-06-17T17:46:52Z,0,0.042,0.887,0.071,0.5396,positive
3,Susurrus,It&#39;s kinda like a video game where you try...,2020-06-10T12:41:45Z,1,0.057,0.698,0.245,0.8821,positive
4,Eugene Scott,Can someone shed some light on why a super mas...,2020-06-06T00:10:48Z,0,0.084,0.760,0.156,0.4215,positive
...,...,...,...,...,...,...,...,...,...
1370,Somillian,Under 301 club.,2015-11-05T22:26:29Z,2,0.000,1.000,0.000,0.0000,neutral
1371,Mythri,First Like LOLOL,2015-11-05T22:24:14Z,2,0.000,0.444,0.556,0.3612,positive
1372,Get Fixed,second,2015-11-05T22:23:58Z,0,0.000,1.000,0.000,0.0000,neutral
1373,ZeroKelvin,Second!,2015-11-05T22:23:57Z,0,0.000,1.000,0.000,0.0000,neutral


In [7]:
def print_top_n_reviews(df,data_column,number_of_rows):

    for index,row in df.nlargest(number_of_rows,data_column).iterrows():
        #print(f"Score: {row[data_column]}, Review: {row[0]}")
        print(f"Score: {row[data_column]}, Comment: {row['comment_text']}")
        

    # video_id = '_tULRch1PRQ'
    # if totalpositivesentiment2 > totalnegativesentiment2 and totalneutralsentiment2:
    #     print('df2 video is overall highly positively acclaimed')
    #     print(f'recommend https://www.youtube.com/watch?v={video_id}')

In [8]:
#below are the 5 most positive comments in 1 video
#how to modify functions to print out 5 most positive videos instead

print_top_n_reviews(text_sentiment,'compound', 5)

Score: 0.9889, Comment: Thank you very much, Phil Plait and the others there at <i>Crash Course</i> Astronomy, for making this video! Wow, the Universe really is amazing! I did know about galaxies before watching this video but I did not know about clusters and superclusters. 100 000 000 000 galaxies! Wow, that’s a pretty big number. It’s only a shame that we only will be able visit so few of them...Anyway, quasars are also splendid, fascinating! Thank you for making this video!<br><br>I do have a question, though. You mentioned that there are a number of ways to classify and group galaxies together, but only talked about two grouping them together based on shape (spiral, elliptical, irregular etc.) and based on energy-output (active, non-active and quasars). So galaxies are really huge, but how massive are there? How do you group them together based on mass? How massive are galaxies?
Score: 0.9842, Comment: OK.<br>So I guess I&#39;m gonna rewatch this whole serie from the beginning be

In [9]:
#according to vader github and jack mckew blog, compound scoring is independent, more useful and so it has more meaningful comments and has more depth than positive column contains degrees of sentiments that is dependent on negative and neutral 
print_top_n_reviews(text_sentiment,'positive', 5)

Score: 1.0, Comment: Lol
Score: 1.0, Comment: Brilliant
Score: 1.0, Comment: awesome
Score: 1.0, Comment: Nice
Score: 1.0, Comment: amazing!!!!!!


In [3]:
#refer to get_top10_sentiment file's loop through all of channel's videos and normalizing every video's comments' unidimensional positives
#then change top10 (by views or likes) dataframe code to top10 (by largest percentages of unidimensional positives based on compound)

#hardcoded... 
#file = f'results/{channel_name_chosen}.json'
import json
file = f'../results/crashcourse.json' 
data = None

with open(file, 'r') as f:
    data = json.load(f)

channel_id, stats = data.popitem()
#print(channel_id)

channel_stats = stats['channel_statistics']

video_stats = stats['video_data']

#analyse video stats
#video_stats.items() value returned after df.popitem()
sorted_vids = sorted(video_stats.items(), key=lambda item: int(item[1]['viewCount']), reverse=True)
#in reverse/descending order so most views in the beginning

stats = []
for vid in sorted_vids:
    video_id = vid[0]
    title = vid[1]['title']
    views = int(vid[1]['viewCount'])
    likes = int(vid[1]['likeCount'])
    
    #sometimes videos have comments disabled so some might not have the key 'commentCount'
    key = "commentCount"
    if key in vid[1].keys():
        comments = int(vid[1]["commentCount"]) 
    else:
        comments = 0

    duration = vid[1]['duration']
    stats.append([video_id, title, views, likes, comments, duration])

df = pd.DataFrame(stats, columns=['video_id', 'title', 'views', 'likes', 'comments','duration'])


In [4]:
df

,video_id,title,views,likes,comments,duration
0,vo4pMVb0R6M,Intro to Psychology: Crash Course Psychology #1,12787803,215402,7144,PT10M54S
1,1wo2TLlMhiw,Conflict in Israel and Palestine: Crash Course...,10942320,182711,0,PT12M53S
2,uBGl2BujkPQ,Introduction to Anatomy & Physiology: Crash Co...,8111652,117613,3152,PT11M20S
3,bO7FQsCcbD8,"The Constitution, the Articles, and Federalism...",7107838,63457,6493,PT13M4S
4,Qymp_VaFo9M,Let's Talk About Sex: Crash Course Psychology #27,6812615,89790,4173,PT11M36S
...,...,...,...,...,...,...
496,wbtpOhIP9Bc,How Does the Healthcare System Work During Out...,38266,1465,9,PT12M18S
497,ftcvaJCKVjs,Marsha P. Johnson and the Stonewall Rebellion:...,37879,2763,69,PT11M59S
498,fXt42GzI1FU,How Can Cooperation End an Outbreak? Crash Cou...,30100,1098,8,PT12M2S
499,V16qfhdIK6M,How to Graduate | How to College | Crash Course,29840,870,51,PT9M32S


In [4]:

overallpositivepercentage = []

#drop/delete row of video that has comments disabled as comments json file of that video does not exist in folder result/crashcourse
#Corey Schafer https://www.youtube.com/watch?v=HQ6XO9eT-fc
filter = df['comments'] == 0
df = df.drop(index=df[filter].index)

for videoID in df['video_id']:

    dataframe = pd.read_json(f'../results/crashcourse/{videoID}.json')
    
    positive = []
    negative = []
    neutral = []
    compound = []
    sentiment = []

    #for line in dataframe['comment_text']:
    for line in range(dataframe.shape[0]): 
#range() method returns total number or length or indexes of 2d array data's rows [0] or data's columns [1]
#use df.shape[0] to only loop through rows while shape[1] gives columns

        comments = dataframe.iloc[line, 1] #iloc returns the value of looped lines/rows of the first[1] column 'Comment' key index
    #print(comments)
        comments_analyzed = analyzer.polarity_scores(comments)
    #print(comments_analyzed)
    #print("{:-<65} {}".format(comments, str(comments_analyzed)))
    
        if comments_analyzed["compound"] >= 0.05:
            eachsentiment = 'positive'
        elif comments_analyzed["compound"] <= -0.05:
            eachsentiment = 'negative'
        else:
            eachsentiment = 'neutral'
    
    #if comments_analyzed["compound"] >= 0:
        #sentiment = 'positive'
    #else:
        #sentiment = 'negative'

#RUN ONCE
        negative.append(comments_analyzed["neg"])
        #negative += comments_analyzed["neg"] is not suitable for this

        positive.append(comments_analyzed["pos"])
        #print(positive)

        neutral.append(comments_analyzed["neu"])
        #print(neutral)

        compound.append(comments_analyzed["compound"])
        #print(compound)

        sentiment.append(eachsentiment)





#add new columns to each dataframe of each top 10 videos

#add new columns and their values to dataframe version of json
    dataframe["negative"] = negative #create and add new column named Negative
    dataframe["neutral"] = neutral
    dataframe["positive"] = positive
    dataframe["compound"] = compound
    dataframe["sentiment"] = sentiment

    #WRONG liner = dataframe.plot.line(subplots=True)
    


    totalrows = len(dataframe['sentiment'])
    #print(f'total rows of dataframe sentiment column = {totalrows}')
    totalpositivesentiment1 = ((dataframe['sentiment'].value_counts()['positive'])/totalrows)*100
    #print(f'overall positive % = {totalpositivesentiment1}')

    overallpositivepercentage.append(totalpositivesentiment1)




    #totalneutralsentiment1 = ((dataframe['sentiment'].value_counts()['neutral'])/totalrows)*100
    
    #totalnegativesentiment1 = ((dataframe['sentiment'].value_counts()['negative'])/totalrows)*100
    
    #print(totalnegativesentiment1+totalneutralsentiment1+totalpositivesentiment1, '%')


df["overallpositivepercentage"] = overallpositivepercentage

#liner = df.plot.line(x='title', y='overallpositivepercentage')

In [4]:
df

,video_id,title,views,likes,comments,duration,overallpositivepercentage
0,vo4pMVb0R6M,Intro to Psychology: Crash Course Psychology #1,12787803,215402,7144,PT10M54S,55.380519
2,uBGl2BujkPQ,Introduction to Anatomy & Physiology: Crash Co...,8111652,117613,3152,PT11M20S,66.337565
3,bO7FQsCcbD8,"The Constitution, the Articles, and Federalism...",7107838,63457,6493,PT13M4S,43.413406
4,Qymp_VaFo9M,Let's Talk About Sex: Crash Course Psychology #27,6812615,89790,4173,PT11M36S,45.098039
5,3ez10ADR_gM,Intro to Economics: Crash Course Econ #1,6512609,93389,4427,PT12M9S,52.084781
...,...,...,...,...,...,...,...
496,wbtpOhIP9Bc,How Does the Healthcare System Work During Out...,38266,1465,9,PT12M18S,77.777778
497,ftcvaJCKVjs,Marsha P. Johnson and the Stonewall Rebellion:...,37879,2763,69,PT11M59S,76.811594
498,fXt42GzI1FU,How Can Cooperation End an Outbreak? Crash Cou...,30100,1098,8,PT12M2S,62.500000
499,V16qfhdIK6M,How to Graduate | How to College | Crash Course,29840,870,51,PT9M32S,54.054054


In [5]:
df = df.sort_values(by=['overallpositivepercentage'], ascending=False)

df.head(10)

,video_id,title,views,likes,comments,duration,overallpositivepercentage
493,jMS-ouZNIzg,Crash Course Office Hours: Geography,50416,684,21,PT1H53S,85.714286
276,Woh63FlFDBk,The Great Migration: Crash Course Black Americ...,253609,8784,29,PT12M40S,82.758621
377,vKaegbtcE00,Arts and Letters of the Harlem Renaissance: Cr...,131673,4777,40,PT12M38S,82.500000
379,ocbAfpjibr4,Ida B. Wells: Crash Course Black American Hist...,129186,5772,34,PT11M34S,82.352941
416,QnVjv-LIgjM,Crash Course Outbreak Science Preview,102913,4179,67,PT4M21S,81.538462
495,j-OMR3h4Isw,Women and the Black Power Movement: Crash Cour...,40371,2235,62,PT11M4S,81.355932
294,7QOTexnD-NE,Frederick Douglass: Crash Course Black America...,224519,8101,41,PT12M19S,78.048780
496,wbtpOhIP9Bc,How Does the Healthcare System Work During Out...,38266,1465,9,PT12M18S,77.777778
378,f14kiGoexVg,The Great Depression: Crash Course Black Ameri...,131093,5683,39,PT11M53S,76.923077
497,ftcvaJCKVjs,Marsha P. Johnson and the Stonewall Rebellion:...,37879,2763,69,PT11M59S,76.811594


In [18]:
def print_top_n_reviews(dfparam, data_column, number_of_rows):

    for index,row in dfparam.nlargest(number_of_rows,data_column).iterrows():
        #print(f"Score: {row[data_column]}, Review: {row[0]}")
        #print(f"Score: {row[data_column]}, Comment: {row['comment_text']}")
        print(f"Score: {row[data_column]}, URL: https://www.youtube.com/watch?v={row['video_id']}")
        


print_top_n_reviews(df, 'overallpositivepercentage', 10)

Score: 85.71428571428571, URL: https://www.youtube.com/watch?v=jMS-ouZNIzg
Score: 82.75862068965517, URL: https://www.youtube.com/watch?v=Woh63FlFDBk
Score: 82.5, URL: https://www.youtube.com/watch?v=vKaegbtcE00
Score: 82.35294117647058, URL: https://www.youtube.com/watch?v=ocbAfpjibr4
Score: 81.53846153846153, URL: https://www.youtube.com/watch?v=QnVjv-LIgjM
Score: 81.35593220338984, URL: https://www.youtube.com/watch?v=j-OMR3h4Isw
Score: 78.04878048780488, URL: https://www.youtube.com/watch?v=7QOTexnD-NE
Score: 77.77777777777779, URL: https://www.youtube.com/watch?v=wbtpOhIP9Bc
Score: 76.92307692307693, URL: https://www.youtube.com/watch?v=f14kiGoexVg
Score: 76.81159420289855, URL: https://www.youtube.com/watch?v=ftcvaJCKVjs


# Accurate recommendation algorithm/process where each video is in sync with it's score

In [11]:
#because video_id has to go hand in hand with its respective positive percentage, there should only be one for loop with a row iteration function that encapsulates the if condition 

for key, value in df.iterrows():
    if value['overallpositivepercentage'] >= 70:
        print(f"Score: {value['overallpositivepercentage']}, URL: https://www.youtube.com/watch?v={value['video_id']}")
            

Score: 85.71428571428571, URL: https://www.youtube.com/watch?v=jMS-ouZNIzg
Score: 82.75862068965517, URL: https://www.youtube.com/watch?v=Woh63FlFDBk
Score: 82.5, URL: https://www.youtube.com/watch?v=vKaegbtcE00
Score: 82.35294117647058, URL: https://www.youtube.com/watch?v=ocbAfpjibr4
Score: 81.53846153846153, URL: https://www.youtube.com/watch?v=QnVjv-LIgjM
Score: 81.35593220338984, URL: https://www.youtube.com/watch?v=j-OMR3h4Isw
Score: 78.04878048780488, URL: https://www.youtube.com/watch?v=7QOTexnD-NE
Score: 77.77777777777779, URL: https://www.youtube.com/watch?v=wbtpOhIP9Bc
Score: 76.92307692307693, URL: https://www.youtube.com/watch?v=f14kiGoexVg
Score: 76.81159420289855, URL: https://www.youtube.com/watch?v=ftcvaJCKVjs
Score: 76.63551401869158, URL: https://www.youtube.com/watch?v=S72vvfBTQws
Score: 76.19047619047619, URL: https://www.youtube.com/watch?v=57xUbch1viI
Score: 76.19047619047619, URL: https://www.youtube.com/watch?v=72ABMa_PuHU
Score: 73.33333333333333, URL: https:/

# Inaccurate recommendation results where scores do not match their actual video ids

In [18]:

for id in df['video_id']:
    for pos in df['overallpositivepercentage']:
        if pos >= 70:
            print(f"Score: {pos}, URL: https://www.youtube.com/watch?v={id}")

Score: 85.71428571428571, URL: https://www.youtube.com/watch?v=jMS-ouZNIzg
Score: 82.75862068965517, URL: https://www.youtube.com/watch?v=jMS-ouZNIzg
Score: 82.5, URL: https://www.youtube.com/watch?v=jMS-ouZNIzg
Score: 82.35294117647058, URL: https://www.youtube.com/watch?v=jMS-ouZNIzg
Score: 81.53846153846153, URL: https://www.youtube.com/watch?v=jMS-ouZNIzg
Score: 81.35593220338984, URL: https://www.youtube.com/watch?v=jMS-ouZNIzg
Score: 78.04878048780488, URL: https://www.youtube.com/watch?v=jMS-ouZNIzg
Score: 77.77777777777779, URL: https://www.youtube.com/watch?v=jMS-ouZNIzg
Score: 76.92307692307693, URL: https://www.youtube.com/watch?v=jMS-ouZNIzg
Score: 76.81159420289855, URL: https://www.youtube.com/watch?v=jMS-ouZNIzg
Score: 76.63551401869158, URL: https://www.youtube.com/watch?v=jMS-ouZNIzg
Score: 76.19047619047619, URL: https://www.youtube.com/watch?v=jMS-ouZNIzg
Score: 76.19047619047619, URL: https://www.youtube.com/watch?v=jMS-ouZNIzg
Score: 73.33333333333333, URL: https:/